In [ ]:
import numpy as np

from keras.datasets import cifar100
from keras.utils import to_categorical
from keras import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dropout, Dense

Für unser Modell nutzen wir den CIFAR100-Datensatz, der über das Modul *keras.datasets* zur Verfügung steht. Dieser besteht aus insgesamt **60000 farbigen Bildern** aufgeteilt in **50000 Trainingsbilder** und **10000 Testbildern**. Die Bilder haben jeweils eine Auflösung von **32x32 Pixeln** und können folgendermaßen geladen werden:

In [ ]:
(x_train, y_train), (x_test, y_test) = cifar100.load_data(label_mode='fine')

Trainings- und Testdaten können über *cifar100.load_data()* geladen werden. Der Datensatz setzt sich aus Kategorien samt Subkategorien zusammen. Über das Argument *label_mode* fordern wir alle Subkategorien an. Da wir mit unserem Modell nur **Früchte** klassizieren möchten, reduzieren wir unsere Daten auf die Kategorien: **Apple, Orange, Pear**. Diese Klassen entsprechen den Labels mit der ID: Apple = 0, Orange = 53, Pear = 57.

In [ ]:
indices_train = []
indices_test = []

# Wir untersuchen, ob es sich bei einem Eintrag der Trainings-Labels um eines der Früchte handelt.
# Falls ja, fügen wir dessen Index einem zuvor initialisiertem Array hinzu.
for i in range(len(y_train)):
    if y_train[i] == 0 or y_train[i] == 53 or y_train[i] == 57:
        indices_train.append(i)

# Selbiges führen wir für alle Test-Labels durch.
for i in range(len(y_test)):
    if y_test[i] == 0 or y_test[i] == 53 or y_test[i] == 57:
        indices_test.append(i)

# Wir reduzieren unsere Trainings- und Test-Labels auf alle die, der Früchte.
y_train = np.array(y_train[indices_train])
y_test = np.array(y_test[indices_test])

# Wir reduzieren unsere Trainings- und Testdaten auf alle die, der Früchte.
x_train = x_train[np.ravel(indices_train)]
x_test = x_test[np.ravel(indices_test)]

# Für die Konvertierung unserer Label-Vektoren in Binäre-Klassenmatrizen, ändern wir alle ursprünglichen
# Trainings- und Test-Labels in die Werte 0-2.
for i in range(len(y_train)):
    if y_train[i] == 0:
        np.put(y_train, i, 0)
    elif y_train[i] == 53:
        np.put(y_train, i, 1)
    elif y_train[i] == 57:
        np.put(y_train, i, 2)

for i in range(len(y_test)):
    if y_test[i] == 0:
        np.put(y_test, i, 0)
    elif y_test[i] == 53:
        np.put(y_test, i, 1)
    elif y_test[i] == 57:
        np.put(y_test, i, 2)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# Wir dividieren für eine Normalisierung jeden unsere Bilderwerte [0;255] in Werte zwischen [0;1].
# Dieser Schritt dient in unsere Modell einer Verbesserung der "Accuracy".
x_train /= 255
x_test /= 255

# Die Konvertierung unser Label-Vektoren in Binäre-Klassenmatrizen wird für unseren Datensatz benötigt.
y_train = to_categorical(y_train, 3)
y_test = to_categorical(y_test, 3)

Im folgenden Schritt definieren wir unser Modell. Wir nutzen für die Klassifizierung unserer Bilder ein **Convolutional Neural Network (CNN)**. Das Modell besteht aus vier *Convolutional Layern* und drei *Pooling Layern*. Unsere Filtergröße beträgt *3 x 3*. Wir verwenden für jede Schicht die *ReĹu*-Aktivierungsfunktion. Nur unser Ausgabeschicht verwendet eine *Softmax*-Aktivierungsfunktionen. Als Error-Funktion kommt für Multiklassifizierung bestimmte *Categorial_Crossentropy*  zum Einsatz. Als Optimierer benutzen wir den stochastischen Optimierer *Adam*.

In [ ]:
# Wir definieren unser Model über einen Stapel von Schichten. Hierzu initialisieren wir unser sequentialles Modell.
model = Sequential()
model.add(Conv2D(96, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(Conv2D(512, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Dropout(0.4))
model.add(Flatten())

model.add(Dense(2048, activation='relu'))
model.add(Dense(4, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['categorical_accuracy'])

Zum Trainieren des Modells übergeben wir der **fit()**-Methode unsere Trainingsdaten und -Labels. Das Modell wird über 5 Epochen trainiert. Wir verwenden unsere Testdaten und -Labels zur Validierung unseres Modells nach jeder Epoche über das Argument *validation_data*. "Loss" und "Accuracy" werden standardmäßig nach jeder Epoche und nach dem Trainieren ausgegeben (verbose default: 1).

In [ ]:
model.fit(x_train, y_train, epochs=5, validation_data=(x_test, y_test))

Um unser Modell für weitere Aufgaben verwenden zu können nutzen wir die **save**-Methode zur Speicherung unseres Modells als Datei.

In [ ]:
model.save("model.h5")